# Goals: 
In this notebook I am attempting to do the following: 

1. Set up an SQL table with sql alchemy. 
2. Use the the Beautiful soup parsers that I have written to parse the html and then place into a database. 

I had to remake one of the tables. 

### First import the libraries 
    

In [9]:
# modules
%matplotlib inline
%config InlineBackend.figure_format='retina'   #this makes it retina style (oh yeah!)
from bs4 import BeautifulSoup
from os.path import join
from pandas import DataFrame
from os import getcwd, listdir
import matplotlib.pyplot as plt
import pandas as pd


#append the path for my own funtions and import them 
import sys
sys.path.append('/Users/Cody/Documents/InsightProject')
from scraper import * 
from data_functions import *


from multiprocessing.pool import ThreadPool as Pool
from validators.url import url as validate_url

In [ ]:
import sqlalchemy as sa
from sqlalchemy import Column, Table, MetaData,create_engine
from sqlalchemy.dialects.mysql import *
from sqlalchemy.sql.expression import insert

### Use sqlalchemy to establish a connection to the database. 
In sql server I created a database called url. I also had to install and configure the SQL database. I am now going to use this sql data base to contain my strings. 

In [ ]:
engine = sa.create_engine('mysql://root:@localhost/url')

In [ ]:
results= engine.execute('show databases').fetchall()

In [ ]:
results

### Create a table to hold the data for the scrapped websites
1. First create a metadata object and bind it to the engine.
2. Then make the table. This table hold an ID, a url (the string), and a medium text 

In [ ]:
metadata =MetaData()
metadata.bind = engine

I used the following commands to create the tables

```python

url =Table('classifier_htmls',metadata,
            Column('urlID', INTEGER, primary_key=True, autoincrement=True),
            Column('url', VARCHAR(60)),
            Column('htmlsText',MEDIUMTEXT),
            Column('htmlsSize', INTEGER))


student = Table('student_htmls',metadata,
            Column('urlID', INTEGER, primary_key=True, autoincrement=True),
            Column('url', VARCHAR(60)),
            Column('htmlsText',MEDIUMTEXT),
            Column('htmlsSize', INTEGER))
            
metadata.create_all(checkfirst=True)

```

In [ ]:
engine.execute('show tables').fetchall()

In [ ]:
engine.execute('describe student_htmls').fetchall()

## Now time to use the beautiful soup functions to start parsing. 

First I will write a function to be called by the multiprocessing pools. 
This function will: 

1. Import the text, and scrape it. 
2. It will calculate the number of strings for each text. 
3. It will then insert the url, text, and the length of the string into the text. 

In [ ]:
eng = create_engine('mysql://root:@localhost/url',pool_size=32)   #create a new engine with 32 connections

In [ ]:
meta = MetaData()
meta.bind = eng

In [ ]:
classifier_htmls = Table('classifier_htmls',metadata,
            Column('urlID', INTEGER, primary_key=True, autoincrement=True),
            Column('url', VARCHAR(60)),
            Column('htmlsText',MEDIUMTEXT),
            Column('htmlsSize', INTEGER))

In [ ]:
#``` python
def parse_text(html):
    """ parses the html and inserts into the database"""
    
    # parste the text and calculate the size
    html_path = '/home/ubuntu/data/html'
    url = html   #get the urginal url
    html = join(html_path, html)
    parsed = scrape_all_text(html)
    size = len(parsed)
    
    # handle the insertion into the table. 
    eng.dispose()
    with eng.connect() as conn:
        # insert the data into the database. 
        ins = classifier_htmls.insert().values(url = url, htmlsText = parsed, htmlsSize=size)
        conn.execute(ins)
#    ```

In [ ]:
html_path = '/home/ubuntu/data/html'
htmls= [html for html in listdir('/home/ubuntu/data/html')]

In [ ]:
print 'the number of entries is %d' %len(eng.execute('select * from classifier_htmls').fetchall())

``` python i = parse_text(htmls[0]) ```

In [ ]:
#I used this script to parse the data. I am going to write a new fuction to scrape t

#```python

def insert_htmls(htmls,threads = 32):
    """inserts multoples"""
    pool = Pool(threads)
    pool.map(parse_text, htmls)
    pool.close()   #```

In [ ]:
#```python 
insert_htmls(htmls) #```

### Add more the data that I previously downloaded to the data base. 

The goal of this part is to repeat what I did, scrape the remaing sites and input them into the sql database.  To do this. 

1. Set the path and file location. 
2. Modify the two fuctions. 
3. Run it. 

In [ ]:
htmls= [html for html in listdir('/home/ubuntu/data/GoGuardianHTMLS-9-17-2015-A')]

In [ ]:
def parse_text(html):
    """ parses the html and inserts into the database"""
    
    # parste the text and calculate the size
    html_path = '/home/ubuntu/data/GoGuardianHTMLS-9-17-2015-A'
    url = html   #get the urginal url
    html = join(html_path, html)
    parsed = scrape_all_text(html)
    size = len(parsed)
    
    # handle the insertion into the table. 
    eng.dispose()
    with eng.connect() as conn:
        # insert the data into the database. 
        ins = classifier_htmls.insert().values(url = url, htmlsText = parsed, htmlsSize=size)
        conn.execute(ins)

In [ ]:
def insert_htmls(htmls,threads = 32):
    """inserts multoples"""
    pool = Pool(threads)
    pool.map(parse_text, htmls)
    pool.close() 

```insert_htmls(htmls)```

In [ ]:
r = engine.execute('Select * from classifier_htmls').fetchall()

In [ ]:
len(r)

In [ ]:
path1 = '/home/ubuntu/data/html'
path2 = '/home/ubuntu/data/GoGuardianHTMLS-9-17-2015-A'
len(listdir(path1)) +len(listdir(path2))

In [ ]:
data = pd.read_sql('Select * from classifier_htmls', con = engine)

In [ ]:
data